In [ ]:
!pip install bert-extractive-summarizer

In [ ]:
!pip install torch transformers numpy scikit-learn


In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2


In [ ]:
!pip install torch   # For PyTorch
!pip install tensorflow   # For TensorFlow (optional)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 603.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline  # For NLP models (text generation, summarization, etc.)
from datasets import load_dataset  # To access Hugging Face datasets
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Ensure NLTK resources are downloaded
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)  # Needed for WordNet lemmatizer

# Load the CNN/DailyMail dataset
try:
    dataset = load_dataset("cnn_dailymail", "3.0.0")

    # Extract train and test data
    train_texts = dataset["train"]["article"][:1000]  # Load only first 100 samples for efficiency
    train_summaries = dataset["train"]["highlights"][:1000]

    test_texts = dataset["test"]["article"][:600]  # Load only first 20 samples for testing
    test_summaries = dataset["test"]["highlights"][:600]

    print(f"Loaded {len(train_texts)} training samples and {len(test_texts)} test samples.")
except Exception as e:
    print(f"Error loading dataset: {e}")

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    """Converts text to lowercase, tokenizes, and applies lemmatization."""
    try:
        tokens = word_tokenize(text.lower())  # Convert to lowercase
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
        return " ".join(lemmatized_tokens)
    except Exception as e:
        print(f"Error processing text: {e}")
        return text  # Return original text in case of error



/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:251: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Loaded 1000 training samples and 600 test samples.


In [ ]:
from sklearn.model_selection import train_test_split

# First, split into train (70%) and temp (30%)
train_texts, temp_texts, train_summaries, temp_summaries = train_test_split(
    train_texts, train_summaries, test_size=0.30, random_state=42)

# Split temp into validation (10%) and test (20%)
val_texts, test_texts, val_summaries, test_summaries = train_test_split(
    temp_texts, temp_summaries, test_size=2/3, random_state=42)  # 2/3 of 30% = 20%



In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
from torch.optim import AdamW
import torch.optim as optim

# Load PEGASUS tokenizer and model
model_name = "google/pegasus-cnn_dailymail"  # Pre-trained PEGASUS model
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Define optimizer and move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() and torch.cuda.memory_allocated() < 1e9 else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8, weight_decay=0.01)

# Training loop
epochs = 100
batch_size = 10

for epoch in range(epochs):
    total_loss = 0
    for i in range(0, len(train_texts), batch_size):
        batch_texts = train_texts[i : i + batch_size]
        batch_summaries = train_summaries[i : i + batch_size]

        optimizer.zero_grad()

        # Tokenize inputs and outputs
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)
        targets = tokenizer(batch_summaries, padding=True, truncation=True, return_tensors="pt").to(device)

        # Forward pass
        outputs = model(input_ids=inputs["input_ids"], labels=targets["input_ids"])
        loss = outputs.loss

        # Backpropagation
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (0) is identical to the `bos_token_id` (0), `eos_token_id` (1), or the `sep_token_id` (None), and your input is not padded.


In [ ]:
import torch
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")

# Extract test data
test_texts = dataset["test"]["article"][:600]  # Load first 600 samples
test_summaries = dataset["test"]["highlights"][:600]

print(f"Loaded {len(test_texts)} test samples.")  # Debugging step

# Now call the function
evaluate_model(model, tokenizer, test_texts, test_summaries)


def evaluate_model(model, tokenizer, test_texts, test_summaries, num_examples=5, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)  # Ensure model is on the correct device

    for i in range(min(num_examples, len(test_texts))):
        text = test_texts[i]
        reference_summary = test_summaries[i]

        # Tokenize input text and move to device
        inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt").to(device)

        # Generate summary using the model
        summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
        generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        print(f"Example {i+1}")
        print(f"Original Text:\n{text}\n")
        print(f"Reference Summary:\n{reference_summary}\n")
        print(f"Generated Summary:\n{generated_summary}\n")
        print("-" * 50)

print(f"Loaded {len(test_texts)} test samples.")
print(f"First sample: {test_texts[0]}")

# Evaluate the model using the CNN/DailyMail dataset
evaluate_model(model, tokenizer, test_texts, test_summaries)


In [ ]:
!pip install rouge-score
import pickle

from rouge_score import rouge_scorer

def evaluate_model(model, tokenizer, test_texts, test_summaries, num_examples=5):
  """
  Evaluates the summarization model on test data and prints the results, including ROUGE scores.

  Args:
    model: The trained summarization model.
    tokenizer: The tokenizer used for the model.
    test_texts: A list of original texts to summarize.
    test_summaries: A list of reference summaries (ground truth).
    num_examples: The number of test examples to evaluate and print.
  """

  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

  for i in range(min(num_examples, len(test_texts))):
      text = test_texts[i]
      reference_summary = test_summaries[i]

      # Generate summary using the model
      inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt").to(device)
      summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
      generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

      print(f"Original Text:\n{text}\n")
      print(f"Reference Summary:\n{reference_summary}\n")
      print(f"Generated Summary:\n{generated_summary}\n")

      # Calculate ROUGE scores
      scores = scorer.score(reference_summary, generated_summary)
      print("ROUGE Scores:")
      for rouge_type, score in scores.items():
          print(f"{rouge_type}: Precision={score.precision:.4f}, Recall={score.recall:.4f}, F1={score.fmeasure:.4f}")

      print("-" * 50)


# Evaluate the model on a small subset of the test data
evaluate_model(model, tokenizer, test_texts, test_summaries)

with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model saved successfully!")


In [ ]:
# New test input based on the AI-powered resume analyzer description
new_text = """
Title: AI-Powered Resume Analyzer
Abstract: A machine learning-based tool that analyzes resumes and provides feedback on improvements.
Problem Statement: Recruiters receive thousands of resumes, making it difficult to manually analyze each one efficiently.
Proposed Methodology: Using NLP and ML algorithms to extract key information from resumes, match them with job descriptions, and provide AI-generated suggestions.
Tech Stack: Python, TensorFlow, Node.js, MongoDB, React
"""

# Add this new input to your test texts
test_texts.append(new_text)
test_summaries.append("An AI-driven resume analysis tool that uses NLP to enhance resume quality by comparing it with job descriptions.")  # A reference summary, optional but good for ROUGE evaluation

# Evaluate the model on the new text
evaluate_model(model, tokenizer, test_texts[-1:], test_summaries[-1:])


In [ ]:
from google.colab import files
files.download('model.pkl')


In [ ]:
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

print("Model loaded successfully!")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
with open("/content/drive/My Drive/model.pkl", "wb") as file:
    pickle.dump(model, file)
